# 생성한 요약문을 제출 형식에 맞도록 변환 및 저장

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT = "/content/drive/MyDrive/"

In [ ]:
sample = pd.read_csv(ROOT+'my_data/220320_kogpt/sample_submission.csv')
sample.head(2)

,id,summary
0,xAqIHdvEZG,NaN
1,zWSR9EQhrG,NaN


In [ ]:
# 데이터 불러오기
answer = pd.read_csv(ROOT+'my_data/220320_kogpt/answer/answer_best.csv')
answer_long = pd.read_csv(ROOT+'my_data/220320_kogpt/beam_search.csv')
answer_short = pd.read_csv(ROOT+'my_data/220320_kogpt/230329_short_token_toppk_strict.csv')

In [ ]:
# 두개로 나뉘어져 있던 요약문을 하나의 DataFrame으로 병합
answer_concat = pd.concat([answer_long, answer_short], axis=0)
answer_concat

,id,summary
0,xAqIHdvEZG,서울시여성가족재단은 세계 어린이날을 맞아 보육·교육기관에서 어린이가 겪는 성차별적 ...
1,zWSR9EQhrG,정부가 코로나19 여파로 직격탄을 맞은 사회적 취약계층과 소상공인 등에 대해 한시적...
2,FdCwnykWkd,창원 경상대병원 노동조합은 간호사들이 의사로부터 폭언과 폭행을 당했다며 노동부 창원...
3,Kh6UuwGxhE,신천지 강제수사 주장에 대해 중앙재난안전대책본부는 코로나19 방역에 부정적이라는 이...
4,yv48O9zTor,부산시가 지하차도 배수펌프 용량을 증설해왔지만 배수펌프 용량이 부족해 폭우에 취약하...
...,...,...
258,wMCwrYmGGK,유 위원은 국내경기 부진이 초래된 원인이 해외투자라는 명목으로 외국으로 나간 경상수...
259,UZt8k9n0Z6,은행이 통지를 받은 시점부터 은행한테 손해배상 책임이 있는 것을 고려할 때 신고가 ...
260,Lg1fQG5DpJ,12세 이하의 어린이의 경우 부모를 통한 가족 내 전파가 학교 관련 전파보다 많았으...
261,UHZEucimy0,김 위원은 정부가 돈을 내려 보내도 지방정부가 매칭을 못하면 사업을 못하게 되지 않...


In [ ]:
# 제출 샘플의 id를 기준으로 merge
merged_answer = pd.merge(sample, answer_concat, how='inner', on='id')

In [ ]:
# 제출 샘플의 기존값 삭제 및 컬럼 이름 조정
merged_answer.drop('summary_x', axis=1, inplace=True)
merged_answer.columns = ['id', 'summary']

In [ ]:
# 결과 확인
merged_answer.head(2)

,id,summary
0,xAqIHdvEZG,서울시여성가족재단은 세계 어린이날을 맞아 보육·교육기관에서 어린이가 겪는 성차별적 ...
1,zWSR9EQhrG,정부가 코로나19 여파로 직격탄을 맞은 사회적 취약계층과 소상공인 등에 대해 한시적...


In [ ]:
merged_answer['id'].duplicated().value_counts()

False    500
Name: id, dtype: int64

In [ ]:
# 로컬에 저장
merged_answer.to_csv(ROOT+'my_data\\220320_kogpt\\answer_mix.csv', encoding='utf-8', index=False)

## long, short 중 하나만 갱신할 때

In [ ]:
# 기존 답안에서 변경된 short 또는 long 답안을 merge
answer_concat = pd.merge(answer, answer_short, how='left', on='id')

In [ ]:
# 새로운 답안에서 NaN인 값을 기존 답안에서 가져옴
for i in range(500):
    if answer_concat.loc[i,'summary_y'] is np.NaN:
        answer_concat.loc[i,'summary_y'] = answer_concat.loc[i,'summary_x']

In [ ]:
# 제출 샘플의 기존값 삭제 및 컬럼 이름 조정
answer_concat = answer_concat.drop('summary_x', axis=1)
answer_concat.columns = ['id', 'summary']

In [ ]:
# 로컬 컴퓨터에 저장
answer_concat.to_csv(ROOT+'my_data/220320_kogpt/answer/toppk_strict.csv', encoding='utf-8', index=False)